In [1]:
from pymongo import MongoClient

client = MongoClient("localhost:27017")
client.list_database_names()

['admin', 'config', 'local', 'udacity']

In [2]:
df = client.udacity
twitter = df.twitter

In [31]:
# Example Tweet:
from pprint import pprint

pprint(twitter.find_one({'user.screen_name':'zyosouzai'}))

{'_id': ObjectId('63e3b55ebeb9905c522f368b'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Sep 02 19:49:03 +0000 2010',
 'entities': {'hashtags': [{'indices': [57, 68], 'text': 'lv25834899'}],
              'urls': [{'expanded_url': None,
                        'indices': [31, 56],
                        'url': 'http://nico.ms/lv25834899'}],
              'user_mentions': []},
 'favorited': False,
 'geo': None,
 'id': 22825235200,
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_user_id': None,
 'place': None,
 'retweet_count': None,
 'retweeted': False,
 'source': '<a href="http://www.nicovideo.jp/" rel="nofollow">ニコニコ動画</a>',
 'text': '【生放送】（声の病持）風来のシレンＴＭＴＡ を開始しました。 http://nico.ms/lv25834899 #lv25834899',
 'truncated': False,
 'user': {'contributors_enabled': False,
          'created_at': 'Wed Aug 04 13:26:07 +0000 2010',
          'description': '最近女装ショタに興味を持ち始めた高校一年生（♂）\r\n\r\nニコ生でシレン放送とかやってるよ',
          'favourites_count': 0,


# Hashtag retweet average

In [34]:
pipeline = [
    {'$unwind': '$entities.hashtags'},
    {'$group': {'_id': '$entities.hashtags.text',
                'retweet_avg': {'$avg': '$retweet_count'}}},
    {'$sort': {'retweet_avg': 1}}
]

In [35]:
[doc for doc in twitter.aggregate(pipeline)][:10]

[{'_id': 'MegaPowers', 'retweet_avg': None},
 {'_id': 'followers', 'retweet_avg': None},
 {'_id': 'euadimito', 'retweet_avg': None},
 {'_id': 'TENSO', 'retweet_avg': None},
 {'_id': 'Echelon', 'retweet_avg': None},
 {'_id': 'realtalk', 'retweet_avg': None},
 {'_id': 'webhistoria', 'retweet_avg': None},
 {'_id': 'whistleblower', 'retweet_avg': None},
 {'_id': 'huchoices', 'retweet_avg': None},
 {'_id': 'otaku', 'retweet_avg': None}]

In [36]:
pprint(twitter.find_one({'retweet_avg':{'$exists':True}}))

None


# Unique hashtags by user

In [27]:
pipeline = [
    {'$unwind': '$entities.hashtags'},
    {'$group': {'_id': '$user.screen_name',
                'unique_hashtags': {'$addToSet': '$entities.hashtags.text'}}},
    {'$sort': {'_id': -1}}
]

In [28]:
[doc for doc in twitter.aggregate(pipeline)][:10]

[{'_id': 'zzj090728', 'unique_hashtags': ['yaplog']},
 {'_id': 'zyosouzai', 'unique_hashtags': ['lv25834899']},
 {'_id': 'zudyezezwe65',
  'unique_hashtags': ['pop', 'all', 'mun', 'data', 'line', 'kl']},
 {'_id': 'zorapataki', 'unique_hashtags': ['zorapataki']},
 {'_id': 'zootcadillac', 'unique_hashtags': ['TwitterJokeTrial']},
 {'_id': 'zolanx', 'unique_hashtags': ['POW']},
 {'_id': 'zjj0607', 'unique_hashtags': ['yaplog']},
 {'_id': 'ziv56ksi', 'unique_hashtags': ['jackpot', 'cazino']},
 {'_id': 'ziplockfresh1', 'unique_hashtags': ['bieberfact']},
 {'_id': 'zingo_champis', 'unique_hashtags': ['blonk']}]

# Push - count the number of tweets for each user

In [37]:
# $push is similar to $addToSet. The difference is that rather than accumulating only unique values 
# it aggregates all values into an array.

# Using an aggregation query, count the number of tweets for each user. In the same $group stage, 
# use $push to accumulate all the tweet texts for each user. Limit your output to the 5 users
# with the most tweets. 
# Your result documents should include only the fields:
# "_id" (screen name of user), 
# "count" (number of tweets found for the user),
# "tweet_texts" (a list of the tweet texts found for the user).  

# Please modify only the 'make_pipeline' function so that it creates and returns an aggregation 
# pipeline that can be passed to the MongoDB aggregate function. As in our examples in this lesson, 
# the aggregation pipeline should be a list of one or more dictionary objects. 
# Please review the lesson examples if you are unsure of the syntax.

# Your code will be run against a MongoDB instance that we have provided. If you want to run this code 
# locally on your machine, you have to install MongoDB, download and insert the dataset.
# For instructions related to MongoDB setup and datasets please see Course Materials.

# Please note that the dataset you are using here is a smaller version of the twitter dataset used in 
# examples in this lesson. If you attempt some of the same queries that we looked at in the lesson 
# examples, your results will be different.

In [55]:
pipeline = [
    # {'$unwind': '$entities.hashtags'},
    {'$group': {'_id': '$user.screen_name',
                'count': {'$sum': 1},
                'tweet_texts': {'$push': '$text'},
                }},
    {'$sort': {'count': -1}},
    {'$limit': 5}
]

In [56]:
[doc for doc in twitter.aggregate(pipeline)]

[{'_id': 'behcolin',
  'count': 8,
  'tweet_texts': ['RT @VouConfessarQue: #VouConfessarQue já aprendi uma matéria inteira poucos minutos antes de uma prova.',
   'RT @TweetGargalhada: Geisy Arruda nos TT? Achei que tinha finalmente sumido, mas pelo jeito a maré trouxe ela de volta!',
   'RT @TweetGargalhada: Geisy Arruda nos TT? Achei que tinha finalmente sumido, mas pelo jeito a maré trouxe ela de volta!',
   'RT @TweetGargalhada: Geisy Arruda nos TT? Achei que tinha finalmente sumido, mas pelo jeito a maré trouxe ela de volta!',
   'RT @TweetGargalhada: Geisy Arruda nos TT? Achei que tinha finalmente sumido, mas pelo jeito a maré trouxe ela de volta!',
   'RT @TweetGargalhada: Geisy Arruda nos TT? Achei que tinha finalmente sumido, mas pelo jeito a maré trouxe ela de volta!',
   'RT @TweetGargalhada: Geisy Arruda nos TT? Achei que tinha finalmente sumido, mas pelo jeito a maré trouxe ela de volta!',
   'volto jah']},
 {'_id': 'mysterytrick',
  'count': 7,
  'tweet_texts': ['昔カンニンGoo